This report provides status information on the most recent endpoints for a hardcoded list of prioritised list of LPAs, or organisations from an input.

The input should be called 'input.csv' and contain one column, 'organisation' that has the organisation codes for the LPAs to be included in the report.

In [1]:
from datetime import datetime as dt
from IPython.display import display
import urllib
import numpy as np
import math
import pandas as pd
import ipywidgets as widgets
import requests
import urllib.parse
import os

In [34]:
# Get input from .csv or use default prioritised LPAs
input_path = './input.csv'
if os.path.isfile(input_path):
    input_df = pd.read_csv(input_path)
    organisation_list = input_df['organisation'].tolist()
    print('Input file found. Using', len(organisation_list), 'organisations from input file.')
else:
    organisation_list = ['local-authority-eng:BUC', 'local-authority-eng:DAC', 'local-authority-eng:DNC',
    'local-authority-eng:GLO', 'local-authority-eng:CMD', 'local-authority-eng:LBH', 'local-authority-eng:SWK',
    'local-authority-eng:MDW', 'local-authority-eng:NET', 'local-authority-eng:BIR', 'local-authority-eng:CAT',
    'local-authority-eng:EPS', 'local-authority-eng:BNE', 'local-authority-eng:GAT', 'local-authority-eng:GRY',
    'local-authority-eng:KTT', 'local-authority-eng:SAL', 'local-authority-eng:TEW', 'local-authority-eng:WBK',
    'local-authority-eng:DST', 'local-authority-eng:DOV', 'local-authority-eng:LIV', 'local-authority-eng:RDB',
    'local-authority-eng:WFT', 'local-authority-eng:NLN', 'local-authority-eng:NSM', 'local-authority-eng:SLF',
    'local-authority-eng:WRL' ]
    print('Input file not found. Using default list of organisations.')

Input file not found. Using default list of organisations.


In [35]:
# Get organisation info
organisation_info_df = pd.read_csv('https://raw.githubusercontent.com/digital-land/organisation-collection/main/data/local-authority.csv')
organisation_info_df.head()
organisation_name_dict = {}
for organisation in organisation_list:
    organisation_code = organisation.split(':')[1]
    organisation_name = organisation_info_df.loc[organisation_info_df['reference'] == organisation_code].iloc[0]['name']
    organisation_name_dict[organisation] = organisation_name

In [36]:
datasette_url = "https://datasette.planning.data.gov.uk/"

def update_dataframe(organisation):
    global result_df  
    if organisation:
        query = f" s.organisation = '{organisation}'"
    else:
        query = f" s.organisation LIKE '%'"
    params = urllib.parse.urlencode({
        "sql": f"""
        select
          e.endpoint_url,
          l.status,
          s.collection,
          group_concat(DISTINCT sp.pipeline) as pipelines,
          s.organisation,
          o.name,
          max(l.entry_date) maxentrydate,
          max(e.entry_date) entrydate,
          e.end_date
        from
          log l
          inner join source s on l.endpoint = s.endpoint
          inner join organisation o on s.organisation=o.organisation
          inner join endpoint e on l.endpoint = e.endpoint
          inner join source_pipeline sp on s.source = sp.source
        where
           {query} and not collection="brownfield-land"
        group by
          l.endpoint,
          l.status
        order by
          l.endpoint,
          s.collection,
          maxentrydate desc
        """,
        "_size": "max"
    })
    
    url = f"{datasette_url}digital-land.csv?{params}"
    df = pd.read_csv(url)
    result_df = df
    return df

def update_dataframe_latest_status(organisation):
    global new_df
    all_endpoints=update_dataframe(organisation)
    new_df=all_endpoints.copy()
    new_df['maxentrydate'] = pd.to_datetime(new_df['maxentrydate'])
    new_df['last_status'] = None
    new_df['last_updated_date'] = None
    new_df['date_last_status_200'] = None
    
    for index, row in new_df.iterrows():
        if index < len(new_df) - 1 and (row['status']!=200 or pd.isna(row['status'])):
            if row['endpoint_url'] == new_df.at[index + 1, 'endpoint_url']:
                new_df.at[index, 'last_status'] = new_df.at[index + 1, 'status']
                new_df.at[index, 'last_updated_date'] = new_df.at[index + 1, 'maxentrydate']   
    
    new_df.drop_duplicates(subset='endpoint_url', keep='first', inplace=True)
    new_df.reset_index(drop=True, inplace=True)
    for index, row in new_df.iterrows():
        if row['last_status'] is not None:
            if row['last_status'] != 200  or row['last_status'] is None:
                filtered_df = all_endpoints[(all_endpoints['endpoint_url'] == row['endpoint_url'] ) & (all_endpoints['status'] == 200)]
                if not filtered_df.empty:
                    new_df.at[index, 'date_last_status_200'] = filtered_df['maxentrydate'].values[0][:19] 
    return new_df

collection_list = ['article-4-direction', 'article-4-direction-area', 'conservation-area', 'conservation-area-document', 'listed-building-outline', 'tree-preservation-order', 'tree-preservation-zone', 'tree']
pipelines_list = ['article-4-direction', 'article-4-direction-area', 'conservation-area', 'conservation-area-document', 'listed-building-outline', 'tree-preservation-order', 'tree-preservation-zone', 'tree', 'tree,tree-preservation-order', 'tree-preservation-order,tree-preservation-zone']
all_orgs_recent_endpoints={}
for organisation in organisation_list:
    recent_endpoints_df = update_dataframe_latest_status(organisation)
    recent_endpoints_df = recent_endpoints_df[recent_endpoints_df['pipelines'].isin(pipelines_list)]
    all_orgs_recent_endpoints[organisation] = recent_endpoints_df


In [37]:
pd.set_option('display.max_colwidth', None)

def compute_cell_colour(status):
    if status == 200:
        return 'background-color: green'
    elif status == 'No endpoint':
        return 'background-color: orange'
    else:
        return 'background-color: red'

rows_list = []
for organisation in organisation_list:
    df = all_orgs_recent_endpoints[organisation]
    if organisation == 'local-authority-eng:BIR': print(df)
    # Revisit this drop duplicates - figure out how we should be handling multiple endpoints for same collection
    # Take the most recent entrydate/maxentrydate?
    df = df.drop_duplicates(subset='pipelines', keep='first')
    try:
        name = organisation_name_dict[organisation]
    except:
        name = organisation
    statuses = {}
    for collection in pipelines_list:
        status = df[df['pipelines'] == collection]['status']
        if ',' in collection:
            datasets = collection.split(',')
            if status.empty or pd.isna(status.values[0]):
                status = 'No endpoint'
            else:
                status = int(status.values[0])
            for i in datasets:
                statuses[i] = status
        else:
            if status.empty or pd.isna(status.values[0]):
                status = 'No endpoint'
            else:
                status = int(status.values[0])
            statuses[collection] = status
    new_row = {'organisation': name}
    new_row.update(statuses)
    rows_list.append(new_row)

output_df = pd.DataFrame(rows_list, columns=['organisation', *collection_list])
output_df.style.applymap(compute_cell_colour, subset=collection_list)

Empty DataFrame
Columns: [endpoint_url, status, collection, pipelines, organisation, name, maxentrydate, entrydate, end_date, last_status, last_updated_date, date_last_status_200]
Index: []


/tmp/ipykernel_9050/3958972033.py:45: FutureWarning: Styler.applymap has been deprecated. Use Styler.map instead.
  output_df.style.applymap(compute_cell_colour, subset=collection_list)


,organisation,article-4-direction,article-4-direction-area,conservation-area,conservation-area-document,listed-building-outline,tree-preservation-order,tree-preservation-zone,tree
0,Buckinghamshire Council,No endpoint,200,200,No endpoint,200,No endpoint,No endpoint,No endpoint
1,Dacorum Borough Council,No endpoint,No endpoint,No endpoint,No endpoint,No endpoint,No endpoint,No endpoint,No endpoint
2,Doncaster Metropolitan Borough Council,No endpoint,200,200,No endpoint,No endpoint,No endpoint,No endpoint,No endpoint
3,Gloucester City Council,No endpoint,200,200,No endpoint,200,No endpoint,No endpoint,No endpoint
4,London Borough of Camden,200,200,200,No endpoint,200,No endpoint,No endpoint,No endpoint
5,London Borough of Lambeth,No endpoint,404,200,No endpoint,200,No endpoint,No endpoint,200
6,London Borough of Southwark,200,200,200,No endpoint,200,200,200,No endpoint
7,Medway Council,No endpoint,200,No endpoint,No endpoint,200,200,200,No endpoint
8,Newcastle City Council,200,200,200,No endpoint,200,200,200,200
9,Birmingham City Council,No endpoint,No endpoint,No endpoint,No endpoint,No endpoint,No endpoint,No endpoint,No endpoint
